# Script to make playing cards

- make a new output directory

In [181]:
import os
import importlib as imp
import shutil
import swg as swg
imp.reload(swg)

<module 'swg' from 'C:\\Users\\micha\\Documents\\InfiniteScripts\\swg.py'>

### create the output directory structure

In [182]:
batchName = "cards"
stage = 'staging/' + batchName + '/'

In [183]:
outf = swg.setCardsOut(batchName)
print(outf)

tempAPT = "templates/singlecard.apt"
tempSHT = "templates/singlecard.sht"
tempMSH = "templates/singlecard.msh"
tempIFF = "templates/singlecard.iff"

output/cards/cards11/


### get the list of images to turn into cards from staging folder
Make the files like this:
- copy the DDS to the output directory
- make the SHT file based on the DDS file name
- make the object IFF file as shared_

In [184]:
def moveDDS(fname, nname):
    shutil.copy(stage + fname, outf + "/texture/" + nname)
    
def makeSHT(fname, nname):
    swg.replace_text_in_file(tempSHT, swg.strSHT(), fname, outf + "/shader/" + nname)

def makeAPT(fname, nname):
    swg.replace_text_in_file(tempAPT, swg.strAPT(), fname, outf + "/appearance/" + nname)

def makeMSH(fname, nname):
    swg.replace_text_in_file(tempMSH, swg.strMESH(), fname, outf + "/appearance/mesh/" + nname)
    
def makeOBJ(fname, nname):
    str1, str2, apt = swg.strOBJ()
    ofile = outf + "/object/tangible/cards/shared_" + nname
    swg.replace_text_in_file(tempIFF, str1, fname, ofile)
    swg.replace_text_in_file(ofile, str2, fname, ofile)
    swg.replace_text_in_file(ofile, apt, fname, ofile)
    
def writeLua(lines, fname):
    file = open(fname,'w')
    for l in lines:
        file.write(l+"\n")
    file.close()
    

In [185]:
cards = os.listdir(stage)
print("cards to process: " + str(len(cards)))
croots = []
oroots = []
luaServerObjects = []
luaLootObjects = []
luaLootObject = []
luaObjects = []
luaCards = []
stringFile = []
for c in cards:
    fn = c.replace('.dds','').lower()
    nname = swg.nname(fn).lower()
    croots.append(c.lower())
    oroots.append(nname)
    moveDDS(c.lower(),nname + '.dds')    
    makeSHT(fn,nname + '.sht')    
    makeOBJ(fn,nname + '.iff')
    makeAPT(fn,nname + '.apt')
    makeMSH(fn,nname + '.msh')
    
    stringFile.append(nname)
    
    luaServerObjects.append('includeFile("../custom_scripts/object/tangible/cards/' + nname + '.lua")')
    
    # objects.lua
    luaObjects.append("-- " + nname);
    luaObjects.append("object_tangible_cards_shared_" + nname + " = SharedTangibleObjectTemplate:new {clientTemplateFileName = \"object/tangible/cards/shared_" + nname + ".iff\"}");
    luaObjects.append("ObjectTemplates:addClientTemplate(object_tangible_cards_shared_" + nname + ", \"object/tangible/cards/shared_" + nname + ".iff\")");
    luaObjects.append("");
    # object file
    luaObject = []
    luaObject.append("-- generated by Wicked's loot machine (python)");
    luaObject.append("-- string name is in cards_n: " + fn);
    luaObject.append("-- string description is in cards_d: " + fn);
    luaObject.append("-- layout: playing card");
    luaObject.append("object_tangible_cards_" + nname + " = object_tangible_cards_shared_" + nname + ":new {}");
    luaObject.append("ObjectTemplates:addTemplate(object_tangible_cards_" + nname + ", \"object/tangible/cards/" + nname + ".iff\")");
    writeLua(luaObject, outf + '/lua/object/tangible/cards/' + nname + '.lua')
    
    # serverobjects.lua
    luaLootObjects.append('includeFile(\"../custom_scripts/loot/items/cards/' + nname + '.lua\")')
    
    # loot object
    luaLootObject = []
    luaLootObject.append("-- generated by Wicked's loot machine (python)");
    luaLootObject.append(nname + " = {");
    luaLootObject.append("minimumLevel = 0,");
    luaLootObject.append("maximumLevel = 0,");
    luaLootObject.append("customObjectName = \"\",");
    luaLootObject.append("directObjectTemplate = \"object/tangible/cards/" + nname + ".iff\",");
    luaLootObject.append("craftingValues = { },");
    luaLootObject.append("customizationStringNames = { },");
    luaLootObject.append("customizationValues = { }");
    luaLootObject.append("}");
    luaLootObject.append("addLootItemTemplate(\"" + nname + "\", " + nname + ")");
    writeLua(luaLootObject, outf + '/lua/lootObjects/' + nname + '.lua')
    
    
# now write out the compiled lists to their object/serverobject files
writeLua(luaObjects, outf + '/lua/object/tangible/cards/objects.lua')
writeLua(luaServerObjects, outf + '/lua/object/tangible/cards/serverobjects.lua')
writeLua(luaLootObjects, outf + '/lua/lootObjects/serverobjects.lua')
writeLua(stringFile, outf + '/strings.txt')







cards to process: 86
